### Fragmenstein demo — Playground (Light)

Fragmenstein is a position-based fragment-merging/placing python3 tool.

<img src="https://github.com/matteoferla/Fragmenstein/raw/master/images/overview.png" width="800" alt="logo">

In its merging/linking operation, under the coordination of the class Victor,
the class Monster finds spatially overlapping atoms and stitches them together (with RDKit),
then the class Igor reanimates (minimises in PyRosetta) them within the protein site restraining the atoms to original positions.
As this compound may not be purchasable, one can use the placement operation to
make a stitched-together molecule based a template.

Fragmenstein can partially work without PyRosetta, as PyRosetta take a few minutes to be installed it's not everyone's cup of tea. Hence this lighter version.

| Name | Colab Link | PyRosetta | Description |
| :--- | :--- | :---: | :--- |
| Pipeline | [![colab demo](https://img.shields.io/badge/Run_full_demo-fragmenstein.ipynb-f9ab00?logo=googlecolab)](https://colab.research.google.com/github/matteoferla/Fragmenstein/blob/master/colab_fragmenstein.ipynb) | &#10004;| Given a template and a some hits, <br>merge them <br>and place the most similar purchasable analogues from Enamine REAL |
| Light | [![colab demo](https://img.shields.io/badge/Run_light_demo-fragmenstein.ipynb-f9ab00?logo=googlecolab)](https://colab.research.google.com/github/matteoferla/Fragmenstein/blob/master/colab_playground.ipynb) | &#10004;| Generate molecules and see how they merge<br>and how a placed compound fairs|

### Operations

1. Generate molecules (as a test of operations)
2. Move molecules for testing and merge molecules via the classes Walton+Monster
3. Search for similars
4. Place a SMILES

### See also
* Fragmenstein:
   [![Documentation Status](https://readthedocs.org/projects/fragmenstein/badge/?version=latest)](https://fragmenstein.readthedocs.io/en/latest/?badge=latest)
[![ github forks matteoferla Fragmenstein?label=Fork&style=social](https://img.shields.io/github/forks/matteoferla/Fragmenstein?label=Fork&style=social&logo=github)](https://github.com/matteoferla/Fragmenstein)
[![ github stars matteoferla Fragmenstein?style=social](https://img.shields.io/github/stars/matteoferla/Fragmenstein?style=social&logo=github)](https://github.com/matteoferla/Fragmenstein)
[![ github watchers matteoferla Fragmenstein?label=Watch&style=social](https://img.shields.io/github/watchers/matteoferla/Fragmenstein?label=Watch&style=social&logo=github)](https://github.com/matteoferla/Fragmenstein)
* [SmallWorld server](https://sw.docking.org/search.html) by Prof John Irwin
* SmallWorld API Python-client: 
   [![Documentation Status](https://readthedocs.org/projects/python-smallworld-api/badge/?version=latest)](https://python-smallworld-api.readthedocs.io/en/latest/?badge=latest)
[![ github forks matteoferla Fragmenstein?label=Fork&style=social](https://img.shields.io/github/forks/matteoferla/Python_SmallWorld_API?label=Fork&style=social&logo=github)](https://github.com/matteoferla/Fragmenstein)
[![ github stars matteoferla Fragmenstein?style=social](https://img.shields.io/github/stars/matteoferla/Python_SmallWorld_API?style=social&logo=github)](https://github.com/matteoferla/Python_SmallWorld_API)
[![ github watchers matteoferla Python_SmallWorld_API?label=Watch&style=social](https://img.shields.io/github/watchers/matteoferla/Python_SmallWorld_API?label=Watch&style=social&logo=github)](https://github.com/matteoferla/Python_SmallWorld_API)
* Hackish widget for JSME in Colab:
[![ github forks matteoferla Fragmenstein?label=Fork&style=social](https://img.shields.io/github/forks/matteoferla/JSME_notebook_hack?label=Fork&style=social&logo=github)](https://github.com/matteoferla/JSME_notebook_hack)
[![ github stars matteoferla Fragmenstein?style=social](https://img.shields.io/github/stars/matteoferla/JSME_notebook_hack?style=social&logo=github)](https://github.com/matteoferla/JSME_notebook_hack)
[![ github watchers matteoferla JSME_notebook_hack?label=Watch&style=social](https://img.shields.io/github/watchers/matteoferla/JSME_notebook_hack?label=Watch&style=social&logo=github)](https://github.com/matteoferla/JSME_notebook_hack)
* [Frankenstein by Mary Shelley](https://en.wikipedia.org/wiki/Frankenstein)

In [ ]:
#@title Installation
#@markdown Press the play button on the top right hand side of this cell
#@markdown once you have checked the settings.
#@markdown You will be notified that this notebook is not from Google:
#@markdown that is normal.

#@markdown Also the code snippet sidebar will appear every time a  `nglview`
#@markdown is added, so don't close it but resize it to a mill wide

#@markdown ### Error reporting
#@markdown To help Matteo improve the code, do you wish to
#@markdown automatically send anonymous error messages?
#@markdown See [here for more](https://github.com/matteoferla/notebook-error-reporter)
report_errors = False #@param {type:"boolean"}

!pip install fragmenstein==0.9.7 smallworld-api notebook-error-reporter
!pip install git+https://github.com/matteoferla/JSME_notebook_hack.git

from google.colab import output
output.enable_custom_widget_manager()

if report_errors:
    from notebook_error_reporter import ErrorServer

    es = ErrorServer(url='https://errors.matteoferla.com', notebook='frag_playground')
    es.enable()

# ------- here for permeance on cell reruns!
from fragmenstein import Walton, display_mols
walton = Walton([])

In [ ]:
#@title Create molecules
#@markdown Running this cells reveals a bunch of widgets allowing 
#@markdown the creation of molecules that will be places in the next cell

#@markdown NB. The iconic JSME smiley-face SMILES input does not work here, 
#@markdown but right-click on the grey menu works though

#@markdown NB2. Adding ünicødè letters (inc. emoji 👾) in the names 
#@markdown seems to work at first, but will cause issues.

#@markdown NB3. If no molecules are added (as happens with `Run all`), two hits are added 
#@markdown from a past panDDA experiment

from jsme_hack import JSMEHack
from IPython.display import display
import ipywidgets as widgets
from rdkit import Chem
from rdkit.Chem import AllChem
from fragmenstein import display_mols
import re, os

# -------------------------------------------------

name_input = widgets.Text(description='Name molecule', value='hit1')
add_button = widgets.Button(description="Add molecule", icon='plus')
clear_button = widgets.Button(description="Discard molecules", icon='trash')
# style does not work in colabs
output = widgets.Output(layout={'border': '1px solid black'})
uploader = widgets.FileUpload(icon='upload')

jsme = JSMEHack()  # outputs to display as is not a widget
grid = widgets.GridspecLayout(1, 4)
grid[0,0] = name_input
grid[0,1] = add_button
grid[0,2] = uploader
grid[0,3] = clear_button
display(grid, output)

def update_output():
    with output:
        output.clear_output()
        if walton.mols == []:
            print('No molecules to show')
        else:
            for mol in walton.mols:
                print(f'{mol.GetProp("_Name")} {Chem.MolToSmiles(mol)}')
        display_mols(walton.mols, useSVG=False)

update_output()

random_names = ['scribbed', 'custom',
                'foo', 'bar', 'baz', 'quack', 'querky',
                'something-ol',
                'something acid',
                'nonexistantene',
                'nonexistant-diene',
                'made-up', 
                'confabulated', 
                'rubbish',
                'trash']

def on_click_add(remove:bool):
    """
    Add molecule
    """
    mol = Chem.MolFromSmiles(jsme.smiles)
    assert mol is not None, f'molecule {jsme.smiles} failed to be parsed'
    AllChem.EmbedMolecule(mol)
    if name_input.value:
        mol.SetProp('_Name', name_input.value)
    else:
        mol.SetProp('_Name', random_names.pop())
    walton.mols.append(mol)
    update_output()

dejaloaded = set()
def observe_upload(*args, **kwargs):
    for new_name in set(uploader.value.keys()) - dejaloaded:  #: str
        forename, ext = os.path.splitext(new_name)
        if ext not in '.mol':
            raise ValueError('Not a mol file')
        dejaloaded.add(new_name)
        mol = Chem.MolFromMolBlock(uploader.value[new_name]['content'])
        if not mol.HasProp('_Name') or not mol.GetProp('_Name'):
            mol.SetProp('_Name', forename)
        walton.mols.append(mol)
    update_output()

def on_click_clear(remove:bool):
    """Remove all mols"""
    walton.mols[:] = []
    update_output()

uploader.observe(observe_upload)
add_button.on_click(on_click_add)
clear_button.on_click(on_click_clear)

In [ ]:
#@title Rototranslate molecules around
#@markdown The following widgets allow basic movements of the molecules.
#@markdown The class that does these movements is Walton, who
#@markdown part of the fragmenstein package but is not involved in any
#@markdown of the merging/placing operatations 
#@markdown —it simply exists for illustrative purposes.
#@markdown (Captain Walton in the novel _Frankenstein_ is the narrator)

#@markdown Notes and caveats:

#@markdown * The GUI is a simple demo and 
#@markdown many Walton methods are unavailable 
#@markdown (align_by_map, create_polygon and
#@markdown all the point based methods such as get_centroid_of_atoms)
#@markdown or are doing a bare minimium. For see Fragmenstein documenation
#@markdown or even tests for examples.

#@markdown * Tooltips _will_ show if you hover over the text of the buttons
#@markdown and keep still for an eternity.

#@markdown * I am not sure why but colours and bond orders do not stick when 
#@markdown in nglview in Colab (which still runs 3.7 and IPython 7!) 
#@markdown when the update command is run on an _already displayed_
#@markdown widget —therefore if you want the colours back, refresh this cell,
#@markdown but be warned the inputs will be reset

#@markdown * The molecules are non properly minimised,
#@markdown therefore beware of proximity bonding by the viewer. 

#@markdown ### Button meanings

from warnings import warn
from fragmenstein.demo import TestSet
from fragmenstein import Walton, Monster, MolNGLWidget
import nglview as nv  # for typehinting
from ipywidgets import TwoByTwoLayout
from IPython.display import clear_output, display, HTML
from functools import partial
from rdkit import Chem, Geometry
from rdkit.Chem import PandasTools
import pandas as pd
from smallworld_api import SmallWorld
from warnings import warn
from threading import Lock
from jsme_hack.rdkit import JSMERDKit

# ========== Namespace pollution ==============
jsme2 = None
monster = None
# =============================================


if len(walton.mols) == 0:
    warn('The previous cell was not used to add `mol`, using mac-x0138 and mac-x0398')
    x0138 = TestSet.get_mol('mac-x0138')
    x0398 = TestSet.get_mol('mac-x0398')
    walton.mols = [x0398, x0138]

# there is the possibility that there are duplicate names... so no dict
names = [mol.GetProp('_Name') for mol in walton.mols]

walton.color_in()
molview:nv.NGLWidget = walton.to_nglview()
# walton.color_in() will have set the mol prop _color 
# which controls the colorValue of the component
# but the update_ball_and_stick oddly does not seem to stick most times
# select
molview.add_selection_signal('clicked_mol', 'clicked_idx')



mol_dropdown = widgets.Dropdown(
    options=names,
    tooltip='The molecule to move',
    description='Molecule to rototranslate:',
    disabled=False,
)

ref_dropdown = widgets.Dropdown(
    options=names,
    description='Ref molecule:',
    disabled=False,
)

atom_idx0_text = widgets.IntText(
                                  value=0,
                                  description='1st atom:',
                                  disabled=False
                              )
atom_idx1_text = widgets.IntText(
                                  value=1,
                                  description='2nd atom:',
                                  disabled=False
                              )
atom_idx2_text = widgets.IntText(
                                  value=1,
                                  description='3rd atom:',
                                  disabled=False
                              )
x_text = widgets.IntText(
                                  value=0,
                                  description='x [Å]',
                                  disabled=False
                              )

y_text = widgets.IntText(
                                  value=0,
                                  description='y [Å]',
                                  disabled=False
                              )

z_text = widgets.IntText(
                                  value=0,
                                  description='z [Å]',
                                  disabled=False
                              )

theta_text = widgets.IntText(
                                  value=0,
                                  description='Angle [°]',
                                  disabled=False
                              )

distance_text = widgets.IntText(
                                  value=0,
                                  description='Distance [Å]',
                                  disabled=False
                              )
axis0_dropdown = widgets.Dropdown(
    options=['x', 'y', 'z'],
    value='x',
    description='1st axis:',
    disabled=False,
)

axis1_dropdown = widgets.Dropdown(
    options=['x', 'y', 'z'],
    value='y',
    description='2nd axis:',
    disabled=False,
)

grid = widgets.GridspecLayout(8, 4)  # row, col
row = 0
grid[row, 0] = mol_dropdown
grid[row, 1] = ref_dropdown

row += 1
grid[row, 0] = atom_idx0_text
grid[row, 1] = atom_idx1_text
grid[row, 2] = atom_idx2_text

row += 1
grid[row, 0] = axis0_dropdown
grid[row, 1] = axis1_dropdown
grid[row, 2] = distance_text

row += 1
grid[row,0] = x_text
grid[row,1] = y_text
grid[row,2] = z_text
grid[row,3] = theta_text

# --------------------------------------------------

def call_walton_method(method, remove:bool):
    possible = dict(mol_idx=names.index(mol_dropdown.value),
                  ref_mol_idx=names.index(ref_dropdown.value),
                  x=x_text.value,
                  y=y_text.value,
                  z=z_text.value,
                  atom_idx=atom_idx0_text.value,
                  axis=axis0_dropdown.value,
                  base_atom_idx=atom_idx0_text.value,
                  pointer_atom_idx=atom_idx1_text.value,
                  atom_idcs=(atom_idx0_text.value, atom_idx1_text.value, atom_idx2_text.value),
                  plane=axis0_dropdown.value+axis1_dropdown.value,
                  theta=theta_text.value,
                  distance=distance_text.value,
                  point=Geometry.Point3D(x_text.value,
                                         y_text.value,
                                         z_text.value),
                  minimize=True,
                  degrees=True,
                  scale=1)
    for k in method.__annotations__:
        if k not in possible:
            assert f'Whats {k} of {method}?'
    method(**{k: possible[k] for k in method.__annotations__ if k != 'return'})
    walton.refresh_nglview(molview)
    molview.center()

def make_button(method, description:str, icon:str='check') -> widgets.Button:
    button = widgets.Button(description=description, icon=icon)
    button.on_click(partial(call_walton_method, method))
    button.tooltip=method.__doc__ if method.__doc__ else 'no docstring'
    return button

row += 1
#@markdown The `duplicate` method copies the main molecule
grid[row, 0] = make_button(walton.duplicate, 'Duplicate', 'copy')
#@markdown The `translate` method translates (moves w/o rotation) the main molecule by (x, y, z)
grid[row, 1] = make_button(walton.translate, 'Translate molecule', 'arrow-up')
#@markdown The `rotate` method rotates the main molecule by theta degrees on the 1st axis
grid[row, 2] = make_button(walton.rotate, 'Rotate molecule', 'rotate-right')
#@markdown The `translate_parallel` method moves the main molecule 
#@markdown on the axis specified by distance &Aring;
#@markdown in the vector parallel to the first atom of the ref mol
#@markdown (base) towards the second atom of the ref mol
#@markdown (ref and main mols can be the same)
grid[row, 3] = make_button(walton.translate_parallel, 'Translate molecule parallel', 'arrow-right')

row += 1
#@markdown Given 3 atoms it will lay the first and third
#@markdown on the primary axis and the second on the plane subtended
#@markdown by the two axes
grid[row, 0] = make_button(walton.flatten_trio, 'Lay 3 atoms on plane', 'align-center')
grid[row, 1] = make_button(walton.atom_on_axis, 'Atom on axis', 'align-center')
grid[row, 2] = make_button(walton.atom_on_plane, 'Atom on plane', 'align-center')
grid[row, 3] = make_button(walton.atom_to_origin, 'Atom to origin', 'align-center')

row += 1
#@markdown Superpose ('align') the other mols onto the selected one
grid[row, 0] = make_button(walton.superpose_by_mcs, 'Superpose onto', 'objects-align-center-horizontal')
#@markdown The `translate_by_point` method translates to the point (x, y, z)
grid[row, 1] = make_button(walton.translate_by_point, 'Translate to point', 'arrow-down')
#@markdown The 'Make merger' button calls the `Monster.combine` method
#@markdown when complete a JSME editor will appear
#@markdown Pressing the 'Place edited' will place it.
#@markdown Search for similar will query the Smallworld server for similars
#@markdown in Enamine REAL and place the one stated by the slider.

mol_listing = widgets.Output(description='SMILES of merger', layout={'border': '1px solid black'})
with mol_listing:
  clear_output()
  display_mols(walton.mols, useSVG=False)

output2 = widgets.Output(description='SMILES of merger', layout={'border': '1px solid black'})
jsme_output = widgets.Output(description='JSME')

        
row += 1
target_text = name_input = widgets.Text(description='Place SMILES', value='')
merge_button = widgets.Button(description='Make merger', 
                              button_style='success',
                              tooltip=walton.__call__.__doc__,
                              icon='calculator')

place_edited_button = widgets.Button(description='Place edited', 
                                     button_style='warning',
                                     tooltip='Place edited',
                                     icon='pencil')
search_button = widgets.Button(description='Search for similars', 
                                     button_style='warning',
                                     tooltip='Search via Smallworld server',
                                       icon='magnifying-glass')

search_show_slider = widgets.IntSlider(min=0,
                                       max=9,
                                      step=1,
                                        description='Show similar')

grid[row, 0] = merge_button
grid[row, 1] = place_edited_button
grid[row, 2] = search_button
grid[row, 3] = search_show_slider


def update_jsme(mol: Chem.Mol):
    with jsme_output:
        clear_output()
        global jsme2
        jsme2 = JSMERDKit(mol)  # type dispatched

def make_merger(remove=False):
    merger:Chem.Mol = walton()
    with output2:
        clear_output()
        print(Chem.MolToSmiles(merger))
    merge_button.button_style = 'success'
    place_edited_button.button_style = 'primary'
    search_button.button_style = ''
    update_jsme(merger)
    with mol_listing:
      clear_output()
      display_mols([*walton.mols, merger], useSVG=False)

merge_button.on_click(make_merger)

def place_edited(remove: bool=False):
    if not jsme2:
        return
    global monster
    monster = Monster([walton.merged])
    monster.place(jsme2.mol, 
             merging_mode='expansion')
    walton.refresh_nglview(molview)
    molview.add_mol(monster.positioned_mol, colorValue='#DC143C')  # crimson
    molview.center()
    with mol_listing:
      clear_output()
      display_mols([*walton.mols, walton.merged, monster.positioned_mol], useSVG=False)
    merge_button.button_style = 'info'
    place_edited_button.button_style = 'info'
    search_button.button_style = 'success'
        
place_edited_button.on_click(place_edited)

sws = SmallWorld()
# this call requires an internet connection
chemical_databases:pd.DataFrame = sws.retrieve_databases()
similars = pd.DataFrame()
previous_search = ''
def search_for_similar(remove=True):
    """
    I tried adding a Lock, but it got messy."""
    global similars
    global previous_search
    with output2:
        if not walton.merged:
            print('Generate merger first')
            return
        if jsme2.smiles:
            wanted_smiles = jsme2.smiles
        else:
            wanted_smiles = Chem.MolToSmiles(walton.merged)
        if wanted_smiles == previous_search:
            print('Already done')
            sliced = similars[['name', 'smiles', 'molecule', 'dist', 'mw']]
            PandasTools.RenderImagesInAllDataFrames(images=True)
            with jsme_output:
                clear_output()
                display(HTML(sliced.to_html()))
            return
        print('Running: be patient. Do not elevator call mash!')
        previous_search = wanted_smiles
        
    similars = sws.search(wanted_smiles,
                               dist=25,
                               length=10,
                               db=sws.REAL_dataset,
                               tolerated_exceptions=())
    if similars is None:
        raise ValueError('An error arose in the search — try later?')
    PandasTools.AddMoleculeColumnToFrame(similars,'smiles','molecule')
    sliced = similars[['name', 'smiles', 'molecule', 'dist', 'mw']]
    PandasTools.RenderImagesInAllDataFrames(images=True)
    with jsme_output:
        clear_output()
        display(HTML(sliced.to_html()))
    target_text.value = sliced.smiles[0]
    monster.place(sliced.molecule[0])
    walton.refresh_nglview(molview)
    molview.add_mol(monster.positioned_mol, colorValue='#DC143C')  # crimson
    molview.center()
    search_button.button_style = 'info'
    
def search_change(*agrs, **kwargs):
    if similars is None or not len(similars):
        return
    monster.place(similars.molecule[search_show_slider.value],
                  custom_map=custom_map)
    walton.refresh_nglview(molview)
    molview.add_mol(monster.positioned_mol, colorValue='#DC143C')  # crimson
    molview.center()
    with mol_listing:
      clear_output()
      display_mols([*walton.mols, walton.merged, monster.positioned_mol], useSVG=False)

search_button.on_click(search_for_similar)
search_show_slider.observe(search_change)


display(mol_listing,
        molview, 
        HTML('<div>Last clicked: atom index <span id="clicked_idx">None</span>'+
             ' of <span id="clicked_mol">None</span><div>'),
        grid,
        jsme_output,
        output2)

custom_text = widgets.Text(description='custom_map', 
                           value=f'{{"{walton.merged.GetProp("_Name")}": {{}}}}')
display(custom_text)

In [ ]:
#@title Advanced mapping

#@markdown The code above may make a bad choice and one may want to give a custom
#@markdown map override.

#@markdown The mapping in the placement can be overridden, by providing a dictionary of 
#@markdown strings (template molecule names) to a dictionary of template atom indices
#@markdown to follow-up indices as described in
#@markdown https://fragmenstein.readthedocs.io/en/latest/doc_custom_mapping.html
#@markdown which details the special cases (e.g. forbidding a pair etc.)

from fragmenstein.branding import divergent_colors
import json



if monster is not None and monster.positioned_mol:
    mol = monster.positioned_mol
else:
    mol = walton.merged


jsme3 = JSMERDKit(mol)
mol_listing2 = widgets.Output(description='SMILES of merger', layout={'border': '1px solid black'})

def update_view(*mols):
  color_series = iter(divergent_colors[len(mols)])
  with mol_listing2:
      clear_output()
      display_mols(mols, useSVG=False, molsPerRow=3, subImgSize=(200,200))
      v = MolNGLWidget()
      for mol in mols:
          colorValue = next(color_series)
          v.add_mol(colorValue=colorValue, mol=mol)
      display(v)
      v.handle_resize()
display(mol_listing2)
edited = jsme3.mol
edited.SetProp('_Name', 'edited')
update_view(*walton.mols, edited)


custom_map = {mol.GetProp('_Name'): {} for mol in walton.mols}

custom_text = widgets.Text(description='custom_map', 
                           value=json.dumps(custom_map))
display(custom_text)

custom_button = widgets.Button(description='Place', icon='calculator', button_style='primary')
display(custom_button)


def on_click_custom_button(remove):
  edited = jsme3.mol
  edited.SetProp('_Name', 'edited')
  # circuitous way to read the string:
  # JSON does not accept nums as keys.
  # but user may have written an invalid string
  escaped = re.sub(r'(?<![\w_.\-"\'])(\d+)(?![\w_.\-"\'])', r'"\1"', custom_text.value)
  custom_map = json.loads(escaped)
  custom_map = {name: {int(k): int(v) 
                            for k, v in custom_map[name].items()} 
                              for name in custom_map}
  print(custom_map)
  monster = Monster(walton.mols)
  monster.place(edited, custom_map=custom_map)
  update_view(*walton.mols, monster.positioned_mol)

custom_button.on_click(on_click_custom_button)
